In [1]:
# Load libraries
import pandas as pd
import altair as alt
import numpy as np


In [2]:
# Step 1: Get total attendees per month (from previous analysis)
fanbase = pd.read_csv("Byte_Datasets/clean_fanbase.csv")
total_attendees = fanbase["Games_Attended"].sum()

# Load stadium data
stadium = pd.read_csv("Byte_Datasets/clean_stadium.csv")

# Get food revenue to calculate monthly attendee distribution
food_revenue = stadium[stadium["Source"] == "Food"].copy()
food_revenue = food_revenue.sort_values("Month")
total_food_revenue = food_revenue["Revenue"].sum()
food_revenue["Revenue_Proportion"] = food_revenue["Revenue"] / total_food_revenue
food_revenue["Estimated_Attendees"] = (food_revenue["Revenue_Proportion"] * total_attendees).round().astype(int)

print(f"Total Attendees: {total_attendees:,}")
print("\nMonthly Attendees:")
print(food_revenue[["Month", "Estimated_Attendees"]])


Total Attendees: 399,799

Monthly Attendees:
    Month  Estimated_Attendees
48      1                 8898
49      2                55669
50      3                52032
51      4                44637
52      5                34541
53      6                24211
54      7                29446
55      8                47246
56      9                40119
57     10                57001
58     11                 5998
59     12                    0


In [3]:
# Step 2: Calculate total ticket revenue per month
# Ticket revenue sources: Lower Bowl, Upper Bowl, Season, Premium

ticket_sources = ["Lower Bowl", "Upper Bowl", "Season", "Premium"]
ticket_revenue = stadium[stadium["Source"].isin(ticket_sources)].copy()

# Group by month and sum all ticket revenue sources
monthly_ticket_revenue = ticket_revenue.groupby("Month")["Revenue"].sum().reset_index()
monthly_ticket_revenue.columns = ["Month", "Total_Ticket_Revenue"]

print("\nMonthly Ticket Revenue (Lower Bowl + Upper Bowl + Season + Premium):")
print(monthly_ticket_revenue)



Monthly Ticket Revenue (Lower Bowl + Upper Bowl + Season + Premium):
    Month  Total_Ticket_Revenue
0       1               1215145
1       2               4939854
2       3               4446603
3       4               3740874
4       5               3017538
5       6               1813746
6       7               2559060
7       8               4207794
8       9               3279504
9      10               4799604
10     11                634966
11     12                     0


In [4]:
# Step 3: Calculate average ticket price per month
# Average Ticket Price = Total Ticket Revenue / Number of Attendees

# Merge attendee data with ticket revenue data
price_analysis = monthly_ticket_revenue.merge(
    food_revenue[["Month", "Estimated_Attendees"]], 
    on="Month"
)

# Calculate average ticket price
# Handle division by zero for months with no attendees
price_analysis["Average_Ticket_Price"] = np.where(
    price_analysis["Estimated_Attendees"] > 0,
    price_analysis["Total_Ticket_Revenue"] / price_analysis["Estimated_Attendees"],
    0
)

print("\nAverage Ticket Price by Month:")
print(price_analysis)

# Overall statistics
valid_months = price_analysis[price_analysis["Estimated_Attendees"] > 0]
print(f"\nOverall Average Ticket Price: ${valid_months['Average_Ticket_Price'].mean():.2f}")
print(f"Highest Average Price: ${valid_months['Average_Ticket_Price'].max():.2f} (Month {valid_months.loc[valid_months['Average_Ticket_Price'].idxmax(), 'Month']})")
print(f"Lowest Average Price: ${valid_months['Average_Ticket_Price'].min():.2f} (Month {valid_months.loc[valid_months['Average_Ticket_Price'].idxmin(), 'Month']})")



Average Ticket Price by Month:
    Month  Total_Ticket_Revenue  Estimated_Attendees  Average_Ticket_Price
0       1               1215145                 8898            136.563835
1       2               4939854                55669             88.736173
2       3               4446603                52032             85.459006
3       4               3740874                44637             83.806573
4       5               3017538                34541             87.361049
5       6               1813746                24211             74.914130
6       7               2559060                29446             86.906880
7       8               4207794                47246             89.061381
8       9               3279504                40119             81.744410
9      10               4799604                57001             84.202102
10     11                634966                 5998            105.862954
11     12                     0                    0              0.

In [5]:
# Step 4: Create visualization of average ticket price by month
alt.data_transformers.disable_max_rows()

# Add month labels
month_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 
               7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
price_analysis["Month_Label"] = price_analysis["Month"].map(month_names)

# Filter out months with no attendees for the visualization
price_analysis_filtered = price_analysis[price_analysis["Estimated_Attendees"] > 0].copy()

# Create line chart with points
line = alt.Chart(price_analysis_filtered).mark_line(
    color='#e74c3c',
    strokeWidth=3
).encode(
    x=alt.X('Month:O', title='Month', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Average_Ticket_Price:Q', title='Average Ticket Price ($)', scale=alt.Scale(zero=False)),
)

points = alt.Chart(price_analysis_filtered).mark_point(
    size=100,
    filled=True,
    color='#e74c3c'
).encode(
    x=alt.X('Month:O', title='Month'),
    y=alt.Y('Average_Ticket_Price:Q', title='Average Ticket Price ($)'),
    tooltip=[
        alt.Tooltip('Month_Label:N', title='Month'),
        alt.Tooltip('Average_Ticket_Price:Q', title='Avg Price', format='$,.2f'),
        alt.Tooltip('Total_Ticket_Revenue:Q', title='Total Revenue', format='$,.0f'),
        alt.Tooltip('Estimated_Attendees:Q', title='Attendees', format=',')
    ]
)

# Add text labels above points
text = alt.Chart(price_analysis_filtered).mark_text(
    align='center',
    baseline='bottom',
    dy=-10,
    fontSize=11
).encode(
    x=alt.X('Month:O'),
    y=alt.Y('Average_Ticket_Price:Q'),
    text=alt.Text('Average_Ticket_Price:Q', format='$,.2f')
)

chart = (line + points + text).properties(
    title='Average Ticket Price by Month',
    width=700,
    height=400
)

chart


alt.LayerChart(...)

In [6]:
# Additional visualization: Bar chart comparison
bar_chart = alt.Chart(price_analysis_filtered).mark_bar(color='#3498db').encode(
    x=alt.X('Month:O', title='Month', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Average_Ticket_Price:Q', title='Average Ticket Price ($)'),
    tooltip=[
        alt.Tooltip('Month_Label:N', title='Month'),
        alt.Tooltip('Average_Ticket_Price:Q', title='Avg Price', format='$,.2f'),
        alt.Tooltip('Total_Ticket_Revenue:Q', title='Total Revenue', format='$,.0f'),
        alt.Tooltip('Estimated_Attendees:Q', title='Attendees', format=',')
    ]
).properties(
    title='Average Ticket Price Distribution by Month',
    width=700,
    height=400
)

bar_chart


alt.Chart(...)

In [7]:
# Show the complete data table for reference
print("\nComplete Monthly Analysis:")
print(price_analysis[["Month", "Month_Label", "Total_Ticket_Revenue", "Estimated_Attendees", "Average_Ticket_Price"]])

# Export to CSV for further analysis
price_analysis.to_csv("Byte_Datasets/average_ticket_price_by_month.csv", index=False)
print("\nData exported to: Byte_Datasets/average_ticket_price_by_month.csv")



Complete Monthly Analysis:
    Month Month_Label  Total_Ticket_Revenue  Estimated_Attendees  \
0       1         Jan               1215145                 8898   
1       2         Feb               4939854                55669   
2       3         Mar               4446603                52032   
3       4         Apr               3740874                44637   
4       5         May               3017538                34541   
5       6         Jun               1813746                24211   
6       7         Jul               2559060                29446   
7       8         Aug               4207794                47246   
8       9         Sep               3279504                40119   
9      10         Oct               4799604                57001   
10     11         Nov                634966                 5998   
11     12         Dec                     0                    0   

    Average_Ticket_Price  
0             136.563835  
1              88.736173  
2     